https://adventofcode.com/2015/day/13

In [ ]:
input = r'''Alice would gain 54 happiness units by sitting next to Bob.
Alice would lose 81 happiness units by sitting next to Carol.
Alice would lose 42 happiness units by sitting next to David.
Alice would gain 89 happiness units by sitting next to Eric.
Alice would lose 89 happiness units by sitting next to Frank.
Alice would gain 97 happiness units by sitting next to George.
Alice would lose 94 happiness units by sitting next to Mallory.
Bob would gain 3 happiness units by sitting next to Alice.
Bob would lose 70 happiness units by sitting next to Carol.
Bob would lose 31 happiness units by sitting next to David.
Bob would gain 72 happiness units by sitting next to Eric.
Bob would lose 25 happiness units by sitting next to Frank.
Bob would lose 95 happiness units by sitting next to George.
Bob would gain 11 happiness units by sitting next to Mallory.
Carol would lose 83 happiness units by sitting next to Alice.
Carol would gain 8 happiness units by sitting next to Bob.
Carol would gain 35 happiness units by sitting next to David.
Carol would gain 10 happiness units by sitting next to Eric.
Carol would gain 61 happiness units by sitting next to Frank.
Carol would gain 10 happiness units by sitting next to George.
Carol would gain 29 happiness units by sitting next to Mallory.
David would gain 67 happiness units by sitting next to Alice.
David would gain 25 happiness units by sitting next to Bob.
David would gain 48 happiness units by sitting next to Carol.
David would lose 65 happiness units by sitting next to Eric.
David would gain 8 happiness units by sitting next to Frank.
David would gain 84 happiness units by sitting next to George.
David would gain 9 happiness units by sitting next to Mallory.
Eric would lose 51 happiness units by sitting next to Alice.
Eric would lose 39 happiness units by sitting next to Bob.
Eric would gain 84 happiness units by sitting next to Carol.
Eric would lose 98 happiness units by sitting next to David.
Eric would lose 20 happiness units by sitting next to Frank.
Eric would lose 6 happiness units by sitting next to George.
Eric would gain 60 happiness units by sitting next to Mallory.
Frank would gain 51 happiness units by sitting next to Alice.
Frank would gain 79 happiness units by sitting next to Bob.
Frank would gain 88 happiness units by sitting next to Carol.
Frank would gain 33 happiness units by sitting next to David.
Frank would gain 43 happiness units by sitting next to Eric.
Frank would gain 77 happiness units by sitting next to George.
Frank would lose 3 happiness units by sitting next to Mallory.
George would lose 14 happiness units by sitting next to Alice.
George would lose 12 happiness units by sitting next to Bob.
George would lose 52 happiness units by sitting next to Carol.
George would gain 14 happiness units by sitting next to David.
George would lose 62 happiness units by sitting next to Eric.
George would lose 18 happiness units by sitting next to Frank.
George would lose 17 happiness units by sitting next to Mallory.
Mallory would lose 36 happiness units by sitting next to Alice.
Mallory would gain 76 happiness units by sitting next to Bob.
Mallory would lose 34 happiness units by sitting next to Carol.
Mallory would gain 37 happiness units by sitting next to David.
Mallory would gain 40 happiness units by sitting next to Eric.
Mallory would gain 18 happiness units by sitting next to Frank.
Mallory would gain 7 happiness units by sitting next to George.'''

In [ ]:
import numpy as np
import xarray as xr
import re
from itertools import *

In [ ]:
line_re = re.compile(r'(?P<name1>[A-Za-z]+) would (?P<gol>gain|lose) (?P<num>[0-9]+) happiness units by sitting next to (?P<name2>[A-Za-z]+).')

names = set()
values = []

for line in input.split('\n'):
  mo = line_re.fullmatch(line)
  assert mo is not None
  name1, name2, gol, num = mo.group('name1', 'name2', 'gol', 'num')
  num = int(num)
  if gol == 'lose':
    num *= -1
  names.add(name1)
  names.add(name2)
  values.append((name1, name2, num))

In [ ]:
names = sorted(list(names))
n_names = len(names)

table = xr.DataArray(np.zeros((n_names, n_names), dtype=int), dims=('name1', 'name2'), coords={'name1': names, 'name2': names})

#for name1, name2, num in values:
#  table.loc['name1':name1, 'name2':name2] = 7
#table.loc['name1':'Alice', 'name2':'Bob'] = 3
#table

In [ ]:
axis_d = {a: b for b, a in enumerate(names)}

table = np.zeros((n_names, n_names), dtype=int)
for name1, name2, num in values:
  table[axis_d[name1], axis_d[name2]] += num
table

array([[  0,  54, -81, -42,  89, -89,  97, -94],
       [  3,   0, -70, -31,  72, -25, -95,  11],
       [-83,   8,   0,  35,  10,  61,  10,  29],
       [ 67,  25,  48,   0, -65,   8,  84,   9],
       [-51, -39,  84, -98,   0, -20,  -6,  60],
       [ 51,  79,  88,  33,  43,   0,  77,  -3],
       [-14, -12, -52,  14, -62, -18,   0, -17],
       [-36,  76, -34,  37,  40,  18,   7,   0]])

In [ ]:
table2 = table + table.T
table2

array([[   0,   57, -164,   25,   38,  -38,   83, -130],
       [  57,    0,  -62,   -6,   33,   54, -107,   87],
       [-164,  -62,    0,   83,   94,  149,  -42,   -5],
       [  25,   -6,   83,    0, -163,   41,   98,   46],
       [  38,   33,   94, -163,    0,   23,  -68,  100],
       [ -38,   54,  149,   41,   23,    0,   59,   15],
       [  83, -107,  -42,   98,  -68,   59,    0,  -10],
       [-130,   87,   -5,   46,  100,   15,  -10,    0]])

In [ ]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def cost(order):
  c = table2[axis_d[order[0]], axis_d[order[-1]]]
  for a, b in pairwise(order):
    c += table2[axis_d[a], axis_d[b]]
  return c

max(((p, cost(p)) for p in permutations(names)), key=lambda pair: pair[1])

(('Alice', 'Bob', 'Mallory', 'Eric', 'Carol', 'Frank', 'David', 'George'), 709)

https://adventofcode.com/2015/day/13#part2

In [ ]:
table3 = np.zeros((9,9), dtype=np.int)
table3[:8, :8] += table2
table3

array([[   0,   57, -164,   25,   38,  -38,   83, -130,    0],
       [  57,    0,  -62,   -6,   33,   54, -107,   87,    0],
       [-164,  -62,    0,   83,   94,  149,  -42,   -5,    0],
       [  25,   -6,   83,    0, -163,   41,   98,   46,    0],
       [  38,   33,   94, -163,    0,   23,  -68,  100,    0],
       [ -38,   54,  149,   41,   23,    0,   59,   15,    0],
       [  83, -107,  -42,   98,  -68,   59,    0,  -10,    0],
       [-130,   87,   -5,   46,  100,   15,  -10,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0]])

In [ ]:
axis_d2 = {**axis_d, 'myself': 8}
axis_d2

{'Alice': 0,
 'Bob': 1,
 'Carol': 2,
 'David': 3,
 'Eric': 4,
 'Frank': 5,
 'George': 6,
 'Mallory': 7,
 'myself': 8}

In [ ]:
def cost2(order):
  c = table3[axis_d2[order[0]], axis_d2[order[-1]]]
  for a, b in pairwise(order):
    c += table3[axis_d2[a], axis_d2[b]]
  return c

max(((p, cost2(p)) for p in permutations([*names, 'myself'])), key=lambda pair: pair[1])

(('Alice',
  'Bob',
  'Mallory',
  'Eric',
  'Carol',
  'Frank',
  'myself',
  'David',
  'George'),
 668)